In [ ]:
import os
import requests
from PIL import Image, ImageEnhance
from io import BytesIO

UNSPLASH_ACCESS_KEY = "uRDplyznSct4Gd8KhAiXD04KtORchjhQOdDQ29cVqG0"
KEYWORD = "portrait"
NR_IMAGINI = 50

def descarca_imagini(keyword, nr_imagini=50, folder='images/normal'):
    os.makedirs(folder, exist_ok=True)
    url = "https://api.unsplash.com/search/photos"
    page = 1
    descarcate = 0

    while descarcate < nr_imagini:
        params = {
            "query": keyword,
            "page": page,
            "per_page": 30,
            "client_id": UNSPLASH_ACCESS_KEY
        }
        response = requests.get(url, params=params).json()
        for img_data in response.get("results", []):
            if descarcate >= nr_imagini:
                break
            img_url = img_data["urls"]["small"]
            img_resp = requests.get(img_url)
            img = Image.open(BytesIO(img_resp.content)).convert("RGB")
            img.save(f"{folder}/{descarcate + 1}.jpg")
            descarcate += 1
        page += 1
    print(f"Descarcate {descarcate} imagini in {folder}")

def aplica_sepia(folder_normal='images/normal', folder_sepia='images/sepia'):
    os.makedirs(folder_sepia, exist_ok=True)
    for img_file in os.listdir(folder_normal):
        img_path = os.path.join(folder_normal, img_file)
        img = Image.open(img_path).convert("RGB")
        sepia_img = aplica_filtru_sepia(img)
        sepia_img.save(os.path.join(folder_sepia, img_file))
    print(f"Filtru sepia aplicat pentru imaginile din {folder_normal}")

def aplica_filtru_sepia(image):
    width, height = image.size
    pixels = image.load()  # accesez pixelii
    for py in range(height):
        for px in range(width):
            r, g, b = pixels[px, py]

            tr = int(0.393 * r + 0.769 * g + 0.189 * b)
            tg = int(0.349 * r + 0.686 * g + 0.168 * b)
            tb = int(0.272 * r + 0.534 * g + 0.131 * b)

            pixels[px, py] = (
                min(255, tr),
                min(255, tg),
                min(255, tb)
            )
    return image

# Executie
descarca_imagini(KEYWORD, NR_IMAGINI)
aplica_sepia()
